## Creating Training Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras

shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()


1122304/1115394 [==============================] - 0s 0us/step


In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts([shakespeare_text])

In [10]:
a = tokenizer.texts_to_sequences(["this is a test", "abcd"])
b = tokenizer.sequences_to_texts(a)

print(a, b)

print(len(tokenizer.word_index), tokenizer.document_count) #num of uniq chars, totol number of docs

[[3, 7, 6, 8, 1, 6, 8, 1, 5, 1, 3, 2, 8, 3], [5, 22, 19, 13]] ['t h i s   i s   a   t e s t', 'a b c d']
39 1


In [13]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 #subtract one to start for 0 index 

In [20]:
## Set training Dataset

train_size = len(encoded) * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [25]:
## chopping dataset into mulitple windows
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead (window is 101)
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)  # by default window is non-overlapping, setting shift to 1 the window will be overlapping. The shift maybe fine tuned